In [1]:
import os
import psycopg2
import psycopg2.extras
from sentence_transformers import SentenceTransformer
import pandas as pd

#  connection to PostgreSQL
connection = psycopg2.connect(
    dbname="your_db_name",
    user="your_username",
    password="your_password",
    host="localhost",
    port="5432"
)


#  embedding model using Hugging Face
model_name = "sentence-transformers/all-mpnet-base-v2"  # 768-dimensional embeddings
embedding_model = SentenceTransformer(model_name)




read = pd.read_csv(r"C:\Users\atul.gupta\cleaned_new_pepsico.csv")

read

new_alis = read[['DB Field Name','Definition']].iloc[:3000]
new_alis['embedding'] = None
print(new_alis)



for index, row in new_alis.iterrows():
    field_dict = {}
    field_name = row['DB Field Name']
    definition = row['Definition']
    field_dict[field_name] = definition
    #print(field_dict)
    embeddings = embedding_model.encode([field_dict]).tolist()
    new_alis.at[index, 'embedding'] = embeddings[0]
    #print(embeddings)
new_alis


with connection.cursor() as cursor:
    for index,row in new_alis.iterrows():
        embedding_vector = row['embedding']
        #print(embedding_vector) 
        
        cursor.execute(
            """
            INSERT INTO test_3k_768(db_field_name,definition,embedding)
            VALUES(%s,%s,%s)
            """,
            (row['DB Field Name'], row['Definition'], embedding_vector)
        )  
print("Done")
connection.commit()

c:\Users\atul.gupta\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                               DB Field Name  \
0        HID_Form_Status__Regen_Ag__Projects   
1     HID_Form_Status__Regen_Ag__Agriculture   
2     HID_Form_Status__Regen_Ag__Commodities   
3                 HID_Forms_Completion_Check   
4                                  Thank_you   
...                                      ...   
2995         MaterialMarketingTargetTypeCode   
2996                                   APLAL   
2997                                   PLNNR   
2998                                   STLAN   
2999                                   STLAL   

                                             Definition embedding  
0     Calculated value of overall form completion fo...      None  
1     Calculated value of overall form completion fo...      None  
2     Calculated value of overall form completion fo...      None  
3     Calculated value of overall form completion fo...      None  
4     Message is typically displayed on the last pag...      None  
...            

In [2]:
import numpy as np
import ast

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_similar_definitions(query, top_n=5):
    query_embedding = embedding_model.encode([query]).tolist()[0]

    with connection.cursor() as cursor:
        cursor.execute("SELECT db_field_name, definition, embedding FROM test_3k_768")
        results = cursor.fetchall()
        

    similarities = []
    for row in results:
        db_field_name, definition, embedding_str = row
        
        
        embedding = ast.literal_eval(embedding_str)  
        
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((db_field_name, definition, similarity))

    
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]
#query = "Footer"
query = ["Footer","asset_status","UpdatedById","MarketUnitName","JobFunctionName","PreferredName","GoLiveDate","CurrencyCode","PayrollEndDate","RequisitionId","DateOfDeath","ProbationPeriodEndDate","ResignationDate","ConfirmationDate","HireTypeCode","TerminationNotificationDate","CorporateCardNumber","ResignationTaiwanDate","LocationTypeCode","GeoZoneCode","PrimaryOwnerId","BusinessUnitName"]
for attribute in query:
    results = search_similar_definitions(attribute, top_n=5)
    print("Attribute_name:",attribute)
    for result in results:
        print(f"Field Name: {result[0]}, Definition: {result[1]}, Similarity: {result[2]}")
    print("\n")
    print("\n")
        


Attribute_name: Footer
Field Name: VBAP__FSH_THEME, Definition: The code defines the theme to which the fashion article belongs, Similarity: 0.28735057536382436
Field Name: XTNDownPaymentAmount, Definition: Represents the down payment amount in the document currency, Similarity: 0.27951894080617984
Field Name: BIC_ZSDPROMO, Definition: Indicates Promotional Sales, Similarity: 0.27491806288098053
Field Name: VBAP__SLCST, Definition: It shows how many times the item has been reviewed, Similarity: 0.27009103866339373
Field Name: XTNFashionThemeCode, Definition: The code specifies the fashion of the Theme, Similarity: 0.26335158096360906




Attribute_name: asset_status
Field Name: STS_ITM, Definition: Current status for the Item, Similarity: 0.5874586395324566
Field Name: CH_ON, Definition: This field used for storing Last Changed date for Asset, Similarity: 0.507723892039825
Field Name: XTNPurchasingDocumentItemStatusIndicator, Definition: Denotes Status of Purchasing Document Item, Simi